In [1]:
import sumolib
import networkx as nx
import csv
import pandas as pd
import numpy as np
import re
from shapely.geometry import LineString
from glob import glob

import matplotlib.pyplot as plt 
%matplotlib inline
plt.rcParams["figure.figsize"] = (15, 10) # (w, h)
import seaborn as sns

from utils import *

# SUMOnet
Loading sumo network and extracting edges, together with their geometries and lengths.

Moreover, we identify and map the self-loops.

In [3]:
sumonet_name = 'milan_road_network'
PATH_TO_NETWORK = '../sumo_simulation_data/'
net = sumolib.net.readNet(PATH_TO_NETWORK +'%s.net.xml' %sumonet_name, withInternal = True)

In [5]:
edge_list = net.getEdges()
edges = [ [edge.getFromNode().getID(), edge.getToNode().getID(), edge.getID(), edge.getLength(), [list(x) for x in edge.getShape()]] for edge in edge_list]

#### edges

In [6]:
for i,e in enumerate(edges):    
    for j, el in enumerate(e[4]):
        l = list(el)
        lon, lat = net.convertXY2LonLat(l[0],l[1])
        e[4][j][0] = lon
        e[4][j][1] = lat

In [7]:
#If I have self loops I don't care, I only take one id
diz_chiavi = dict()
for i,e in enumerate(edges):
    if e[2].startswith(":"):
        #print(e[2])
        arr = re.split(':|_', e[2]) 
        arr = arr[1:]
        #print(arr)
        ide = "_".join(arr[:-1])
        #print(ide)
        #print("_____________")
        diz_chiavi[str(e[2])] = ide
        edges[i][2] = ide

In [8]:
df = pd.DataFrame(edges, columns =['from', 'to', 'id', 'length', 'geometry'])
df.head()

,from,to,id,length,geometry
0,1003229693,1003229693,1003229693,15.95,"[[9.15910578799311, 45.46666019292048], [9.159..."
1,1003229693,1003229693,1003229693,16.65,"[[9.15910578799311, 45.46666019292048], [9.159..."
2,1003229693,1003229693,1003229693,4.19,"[[9.15910578799311, 45.46666019292048], [9.159..."
3,1003229693,1003229693,1003229693,8.12,"[[9.15909187022394, 45.46662384750193], [9.159..."
4,1003229693,1003229693,1003229693,18.20,"[[9.159154494267845, 45.46660341757732], [9.15..."


In [9]:
df = df.drop_duplicates(subset=['from', 'to', 'id'], keep='first')
df = df.reset_index(drop=True)
df.head()

,from,to,id,length,geometry
0,1003229693,1003229693,1003229693,15.95,"[[9.15910578799311, 45.46666019292048], [9.159..."
1,1003229750,1003229750,1003229750,4.15,"[[9.150211893006958, 45.46699015679695], [9.15..."
2,1004444456,1004444456,1004444456,9.05,"[[9.140133240658752, 45.46382626552388], [9.14..."
3,1006057388,1006057388,1006057388,9.84,"[[9.148065807316762, 45.45865983251766], [9.14..."
4,1006191585,1006191585,1006191585,8.69,"[[9.163553906931327, 45.461937105599596], [9.1..."


In [10]:
df.loc[df['from'] == df['to'],['length']] = 0

In [11]:
df.head()

,from,to,id,length,geometry
0,1003229693,1003229693,1003229693,0.0,"[[9.15910578799311, 45.46666019292048], [9.159..."
1,1003229750,1003229750,1003229750,0.0,"[[9.150211893006958, 45.46699015679695], [9.15..."
2,1004444456,1004444456,1004444456,0.0,"[[9.140133240658752, 45.46382626552388], [9.14..."
3,1006057388,1006057388,1006057388,0.0,"[[9.148065807316762, 45.45865983251766], [9.14..."
4,1006191585,1006191585,1006191585,0.0,"[[9.163553906931327, 45.461937105599596], [9.1..."


In [12]:
df.loc[16043, 'geometry']

[[9.189784609864201, 45.4767316589976], [9.188606854050814, 45.47695341835128]]

In [25]:
df['geometry'] = df['geometry'].apply(LineString)

In [13]:
df.loc[16043, 'geometry']

[[9.189784609864201, 45.4767316589976], [9.188606854050814, 45.47695341835128]]

In [14]:
df.loc[df['from'] == df['to'],['geometry']] = None

In [15]:
df.head()

,from,to,id,length,geometry
0,1003229693,1003229693,1003229693,0.0,None
1,1003229750,1003229750,1003229750,0.0,None
2,1004444456,1004444456,1004444456,0.0,None
3,1006057388,1006057388,1006057388,0.0,None
4,1006191585,1006191585,1006191585,0.0,None


In [19]:
df.to_csv("edges.csv", index = False, header = False)

# Network Loading
Creating a NetworkX MultiDiGraph to host our edges, with length and geometry as attributes.

In [32]:
G = nx.MultiDiGraph()

In [33]:
df = pd.read_csv("edges.csv", header=None, names=['from', 'to', 'id', 'length', 'geometry'])

In [34]:
import shapely.wkt

for idx, row in df.iterrows():
    #print(row['from'], row['to'], row['id'], row['length'], row['geometry'])
    if row['from'] != row['to']:
        G.add_edge(row['from'], row['to'], 
                   key = str(row['id']), 
                   length = row['length'], 
                   geometry = row['geometry'])
print(len(G.edges))

10632


In [35]:
with open("edges.csv") as f:
    for l in f:
        l = l.rsplit(",")
        #l[4] = l[4].replace("\n","")
        if l[4] != '':
            print(l[4])
            #G.add_edge(l[0],l[1], key = str(l[2]), length = l[3], geometry = l[4] )

In [36]:
x = df[df.duplicated(['from','to'], keep=False)]
x.sort_values(by=['from','to'], ascending=False)

,from,to,id,length,geometry
15289,cluster_2887674292_615041494_615041496,2888859480,48426430#0,8.03,LINESTRING (9.142189899809894 45.4832761696058...
15724,cluster_2887674292_615041494_615041496,2888859480,685615731,0.20,LINESTRING (9.142317219790248 45.4833272276646...
13279,cluster_283056705_31774774,53722819,285857310,14.80,LINESTRING (9.17489847316632 45.47877852485533...
15391,cluster_283056705_31774774,53722819,4889200#0,14.42,LINESTRING (9.174913031635608 45.4788936275254...
12727,cluster_2817355944_2817355945,2805748828,277205491,0.20,LINESTRING (9.158082478933707 45.4666829465025...
...,...,...,...,...,...
10966,1555289199,cluster_1279803167_31333372,267035795#0,16.94,LINESTRING (9.167043301045778 45.4981965010495...
7724,1542124785,cluster_1542124815_21226259,140836955,14.76,LINESTRING (9.150868407326577 45.4598029182604...
10826,1542124785,cluster_1542124815_21226259,266546703,12.04,LINESTRING (9.150816856560647 45.4598011860957...
7538,1281605730,cluster_102231613_1281605663,112805956#0,12.06,LINESTRING (9.219442447908275 45.4485608684963...


In [37]:
G.get_edge_data('cluster_2887674292_615041494_615041496','2888859480')

{'48426430#0': {'length': 8.03,
  'geometry': 'LINESTRING (9.142189899809894 45.48327616960582, 9.142199507587728 45.4832804782039, 9.14228682374412 45.483299541950274)'},
 '685615731': {'length': 0.2,
  'geometry': 'LINESTRING (9.142317219790248 45.483327227664674, 9.142319393051716 45.48332641485238)'}}

In [38]:
G.edges(data=True)

OutMultiEdgeDataView([('3396938718', '1157980324', {'length': 31.36, 'geometry': 'LINESTRING (9.185260027884443 45.46534232989504, 9.185271037671468 45.465345012395645, 9.185563924612259 45.46552573094969)'}), ('3396938718', 'cluster_1766730093_4060097450', {'length': 57.95, 'geometry': 'LINESTRING (9.185219148068553 45.46528073802844, 9.185423748456767 45.46483043651536, 9.185447646589582 45.46478458165179)'}), ('1157980324', '3396938718', {'length': 30.11, 'geometry': 'LINESTRING (9.185536876790803 45.46554737774415, 9.185250142269256 45.46537042964413, 9.185246557617468 45.46536953533841)'}), ('2820985528', '1843088178', {'length': 31.22, 'geometry': 'LINESTRING (9.188522875928431 45.46239567141828, 9.188522850949012 45.4626184510538, 9.188522799536477 45.462641404187266, 9.188522789223564 45.4626766888906)'}), ('2820985528', '2820985527', {'length': 23.86, 'geometry': 'LINESTRING (9.188596160791796 45.4623163399196, 9.188612657716513 45.462315232537485, 9.188901347852763 45.4623132

# CO2 weights
Loading the results from each simulation and adding the CO2 emissions of each one as a new attribute on the edges of the network.

#### Co2 from all simulations

In [28]:
import json

navigator = 'OSM'


if navigator == 'OSM':
    f = open('../results/osm/dict_exps_filename.json')
    dict_exps_filename = json.load(f)
    DATA_PATH = '../sim_outputs/exps_osm/'
elif navigator == 'TT':
    f = open('../results/tomtom/dict_exps_filename_tomtom.json')
    dict_exps_filename = json.load(f)
    DATA_PATH = '../sim_outputs/exps_tomtom/'
else:
    raise ValueError('not a valid navigator.')


# creating dict of dict with only the repetition '0' for each scenario
map__navig__scenario__filename = {}
map__navig__scenario__filename[navigator] = {int(key.split('_')[0]): value['0'] for key,value in dict_exps_filename.items()}


for c_scenario, c_file_name in sorted(map__navig__scenario__filename[navigator].items()):
    
    # loading emissions data
    c_df = pd.read_csv(glob(DATA_PATH+c_file_name)[0])
    c_df['edge_id'] = c_df['edge_id'].map(diz_chiavi).fillna(c_df['edge_id'])
    c_df = c_df.groupby("edge_id", as_index = False).sum()
    co2_dict = dict(zip(c_df['edge_id'], c_df['total_co2']))
    
    # adding emissions as edge attribute
    attr_name = 'co2__'+str(c_scenario)+navigator.lower()
    for f, t, k, data in G.edges(data=True, keys=True):
        if k in co2_dict: 
            i += 1
            data[attr_name] = co2_dict[k]
        else:
            data[attr_name] = 0

In [ ]:
# dict of dict with all the results (one vector of co2 per repetition) for each scenario

navigator = 'OSM'


if navigator == 'OSM':
    f = open('../results/osm/dict_exps_filename.json')
    dict_exps_filename = json.load(f)
    DATA_PATH = '../sim_outputs/exps_osm/'
elif navigator == 'TT':
    f = open('../results/tomtom/dict_exps_filename_tomtom.json')
    dict_exps_filename = json.load(f)
    DATA_PATH = '../sim_outputs/exps_tomtom/'
else:
    raise ValueError('not a valid navigator.')
    

map__scenario__results = {}
map__scenario__repetition__edge__results = {}
for c_scenario, c_dict_results in dict_exps_filename.items():
    c_scenario_number = int(c_scenario.split('_')[0])
    c_list_scenario_results = []
    for c_rep, c_file in c_dict_results.items():
        c_df = pd.read_csv(glob(DATA_PATH + c_file)[0])
        #print(c_df)
        c_df['edge_id'] = c_df['edge_id'].map(diz_chiavi).fillna(c_df['edge_id'])
        c_df = c_df.groupby("edge_id", as_index = False).sum()
        co2_dict = dict(zip(c_df['edge_id'], c_df['total_co2']))
        map__scenario__repetition__edge__results[c_scenario_number][c_rep] = co2_dict
        
        c_list_scenario_results.append(list(c_df['total_co2']))
    map__scenario__results[c_scenario_number] = c_list_scenario_results

    
# dict with mean co2 per road across the 10 repetitions
map__scenario__mean_results = {scenario: np.array(pd.DataFrame(vec_res).mean()) for scenario, vec_res in map__scenario__results.items()}

# Number of vehicles
Loading the number of vehicles (on each edge) from each simulation and adding it as a new attribute to the network.

In [ ]:
PATH_TO_DATA = '../sim_outputs/'

list_sims = [
    'osm_0_dua_100_30_03_14_35_41',
    'osm_20_dua_80_30_03_14_36_47',
    'osm_40_dua_60_30_03_14_40_15',
    'osm_60_dua_40_30_03_15_14_48',
    'osm_80_dua_20_30_03_15_15_28',
    'osm_100_dua_0_30_03_15_17_13'
]

for c_sim in list_sims:
    # loading emissions data
    #print(glob.glob(PATH_TO_DATA + c_sim + "/co2_edge_*")[0])
    df_vehicles = pd.read_csv(glob.glob(PATH_TO_DATA + c_sim + "/vehicles_edge*")[0])
    df_vehicles['edge_id'] = df_vehicles['edge_id'].map(diz_chiavi).fillna(df_vehicles['edge_id'])
    df_vehicles = df_vehicles.groupby("edge_id", as_index = False).sum()
    vehicles_dict = dict(zip(df_vehicles['edge_id'], df_vehicles['count']))
    
    # adding emissions as edge attribute
    attr_name = 'n_vehicles__'+c_sim
    for f, t, k, data in G.edges(data=True, keys=True):
        if k in vehicles_dict: 
            i += 1
            data[attr_name] = vehicles_dict[k]
        else:
            data[attr_name] = 0

# Topological betweenness
Computing the topological edge betweenness centrality of each edge of the network and adding it as a new attribute to the edges of the network.

In [ ]:
eb = nx.edge_betweenness_centrality(G, normalized=True, weight='length')

In [ ]:
eb

In [32]:
nx.set_edge_attributes(G, eb, 'betweenness_centrality')

In [ ]:
G.edges(data=True, keys=False)

## Saving the network as .graphml file

In [29]:
### saving the network
nx.write_graphml(G, PATH_TO_NETWORK + sumonet_name + "__with_attributes.graphml")